# Lab 04. Homework: Convolution Layer 없애기


In [ ]:
import os
import copy
from os.path import join

import warnings
warnings.filterwarnings('ignore')

## Convolutional Neural Network
이미지를 분류하는 모델을 만들어 보겠습니다.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
torch.backends.cudnn.deterministic=True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda
print(device)

### 데이터셋 불러오기

In [ ]:
# Dataset 다운로드
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# print(type(trainset))

## CNN 정의하기: Convolution Layer 없애기
class Net() 부분만 수정해 모델을 돌려보세요!
> 오직 fully connected layer (=nn.Linear) 3개만을 가지고 전체 test 데이터에 대해서 모델의 성능 확인 <br>
> Hint: fully connected layer의 input 차원 수정, view 함수 차원 수정

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # ============= 수정 =============
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # ============= 수정 =============

    def forward(self, x):
        # ============= 수정 =============
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # ============= 수정 =============
        return x

net = Net().to(device)

### 손실함수와 optimizer 정의하기

In [ ]:
# 손실함수와 optimizer 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### CNN 학습하기

In [ ]:
# CNN 학습
for epoch in range(2):  # 시간 관계상 2 epochs만 학습한다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # data => [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # parameter의 gradient 초기화
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

### 모델 저장하기

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

### 전체 test 데이터에 대해 accuracy 계산

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

### Class 별 모델 평가

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))